# Regresión lineal concisa.

En el notebook anterior, vimos un ejemplo de como implementar una red neuronal desde cero. Sin embargo, hacer esto es una mala idea. La principal razón por la que es una mala idea, es que muchas de las cosas que hicimos consisten en "reinventar la rueda". Hay bibliotecas que ya tienen herramientas para hacer lo que ya hicimos. Además, nuestra implementación puede no ser la más eficiente. Es decir: la implementación usada puede generar tiempos de espera que podrían ser evitados si nuestro código estuviera implementado de manera distinta. Por esta razón, es siempre recomendable usar las bibliotecas preexistentes.

Recordemos que el ejemplo anterior estaba pensado para que le perdamos el miedo a las bibliotecas preexistentes, para que entendamos como funcionan y para aprender a implementar cosas nuevas (si llegamos a necesitarlo)

Veamos entonces como implementariamos todo lo anterior haciendo uso de la biblioteca ``pytorch``


## Datos sintéticos


In [ ]:
import numpy as np
import torch
from torch.utils import data

Misma función que usamos en el NB anterior

In [ ]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

## Cargando nuestros datos

En este caso, podemos enviar nuestros datos diferentes metodos preexistentes de `pytorch` para generar nuestro minilotes.

Además podemos pedir nos mezcle nuestros datos o que los deje tal cual


In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [ ]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

Queremos ver como se generan nuestros minilotes. Para esto debemos poder imprimierlos por pantalla. A diferencia de la implementación anterior, el método `DataLoader`, no genera un iterable, por esto debemos convertirlo en uno y recorrerlo segun necesitemos


In [ ]:
next(iter(data_iter))

[tensor([[-0.6694, -0.3256],
         [ 0.1160,  0.6049],
         [ 1.4336,  1.6383],
         [ 1.5785,  1.7636],
         [ 0.4943,  0.2662],
         [ 0.4037,  0.1017],
         [-1.1489,  0.8545],
         [-0.2937,  1.2940],
         [-0.2864, -0.0559],
         [-0.5227,  0.9123]]), tensor([[ 3.9669],
         [ 2.3659],
         [ 1.5006],
         [ 1.3614],
         [ 4.2735],
         [ 4.6633],
         [-0.9977],
         [-0.7902],
         [ 3.8183],
         [ 0.0464]])]

## Definiendo el modelo

A continuación presentamos la versión concisa de nuestro modelo, luego la discutiremos.


In [ ]:
# `nn` = neural networks, redes neuronales
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

La clase `Sequiential` define todas las capas a aplicar de manera secuencial en nuestro modelo. Por ahora, como trabajamos con regresión lineal solo usaremos una capa. Sin embargo esta capa es lo que se llama una capa *totalmente conectada*. Es decir, esta representada por una matriz que aplica sobre vector de features. Al aplicar esta matriz encontramos la salida de nuestra neurona. En este caso, este tipo de capas se las conoce como `Linear` y reciben como entrada `(<numero_de_entradas>, <numero_de_salidas>)`. Para nuestro modelo, esto son nuestras 2 features y nuestra etiqueta.


¿Que es una capa con lineal?
------------------


una capa lineal o completamente conectada es la forma más básica de una red neuronal. Cada entrada influencia a cada salida de acuerdo a los pesos. Si nuestro modelo tiene $m$ entradas y $n$ salidas, la matriz de pesos sera $m \times n$. De igual modo el vector de sesgos o bias tendra dimensión $n$

In [ ]:
import torch

lin = torch.nn.Linear(2, 1)
x = torch.rand(1, 2)
print('entrada:')
print(x)

print('\n\nPesos y parametros:')
for param in lin.named_parameters():
    print(param)

y = lin(x)
print('\n\nsalida')
print(y)

# Al hacer la multiplicacion matricial correspondiente obtenemos nuesta salida.
x @ lin.weight.T + lin.bias

entrada:
tensor([[0.7730, 0.8338]])


Pesos y parametros:
('weight', Parameter containing:
tensor([[ 0.0586, -0.0287]], requires_grad=True))
('bias', Parameter containing:
tensor([0.1773], requires_grad=True))


salida
tensor([[0.1987]], grad_fn=<AddmmBackward0>)


tensor([[0.1987]], grad_fn=<AddBackward0>)

## Inicialización de parametros de nuestro modelo.

Por lo general, los frameworks prexistentes tienen implementaciones por defecto para inicializar los parámetros. Sin embargo, queremos iniciarlos de manera similar a la anterior.

Para ellos accedemos a la primera (y única capa) usando `net[0]`. Luego accedemos a los pesos y los sesgos con `weight.data` and `bias.data`. Finalmente rellenamos los valores con lo que teníamos pensado usar.


In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## Definiendo la función de pérdida


In [ ]:
loss = nn.MSELoss()

## Definiendo el algoritmo de optimización


La principal diferencia con lo que hicimos antes, es solamente debemos pasarle a nuestro `SDG`, los parametros a optimizar. El resto de los detalles ya son manejados por la implementación de `pytorch`. En este caso también estamos pasando la tasa de aprendizaje, pero la clase `SGD` de `pytorch` ya incluye un valor por defecto.

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

Un optimizador en `torch` tiene por defecto una serie de métodos. Sin embargo ahora mismo solo nos interesan 2 de ellos, pues son los que más usaremos.

* `Optimizer.step`
  > Este es el método es el que propiamente aplica el algoritmo SGD, o cualquier otro algoritmo que fueramos a implementar.
* `Optimizer.zero_grad`
  > Por defecto, `Optimizer` suma los sucesivos gradientes calculados. Esto hace que al principio de cada época de el entrenamiento, debamos setear el gradiente en 0. Es por esto que este método existe dentro de la clase `Optimizer`

## Entrenamiento

Hasta aquí veníamos reduciendo lineas de código de manera impresionante. Sin embargo, nuestro ciclo de entrenamiento será casi identico a lo que habíamos visto antes.
* Repetimos hasta concluir
    * Calculamos la función de pérdida
    * Calculamos el gradiente con minilotes 
    * Actualizamos los parámetros. 


In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000197
epoch 2, loss 0.000094
epoch 3, loss 0.000094


In [ ]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0002, 0.0004])
error in estimating b: tensor([0.0003])


Hasta aquí hemos trabajado con el problema de la regresión. Sin embargo, muchas veces lo que deseamos es clasificar segun clases discretas. De hecho, más adelante veremos que los grandes logros de las redes neuronales son en el area de clasificación. Para esto, a continuación hablaremos de Regresión Softmax y su aplicación en clasificación.